In [3]:
# imports
# Required libraries to be installed: Pillow, OpenCV
import cv2 as cv
import numpy as np
import glob
import os
import pickle
import matplotlib.pyplot as plt
from IPython.display import clear_output, Image, display
import PIL.Image
import io

In [4]:
# helper functions
def show_image(a, fmt='jpeg'):
    if(len(a.shape)) == 3:
        a = cv.cvtColor(a, cv.COLOR_BGR2RGB)
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))
    
def read_image(index, source, resize):
    image = cv.imread(source[index])
    return cv.resize(image, (0, 0), fx=resize, fy=resize)

In [5]:
# I tried with multiple images in the beginning
task1 = glob.glob(os.path.join('./', "training_data/Task1/*.jpg")) 
task2 = glob.glob(os.path.join('./', "training_data/Task2/*.mp4")) 
task2_modified = glob.glob(os.path.join('./', "Modified/Task2/*.mp4")) 
# print(task1_images)

In [6]:
img1 = read_image(3, task1, 1)
# show_image(img1)

In [7]:
# This function takes the video path and returns the a list of frames.
def read_frames(video_path, only_table=True):
    frames = []
    cap = cv.VideoCapture(video_path)  
    if cap.isOpened() == False: 
        print("Error opening video stream or file") 
        return frames
    
    while cap.isOpened():  
        ret, frame = cap.read() # Read the frame
        if ret is True:
            if (only_table):
                frames.append(find_table(frame))
            else:
                frames.append(frame)
        else:
            break
    cap.release()
    return frames

In [136]:
# DEPRECATED
# # def find_table(frame, show_results=True):
# #     low_green = (55, 180, 50)
# #     high_green = (75, 255, 255)
# #     frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
# #     # get the mask for green
# #     mask_table_hsv = cv.inRange(frame_hsv, low_green, high_green)

# #     kernel = np.ones((15, 15), np.uint8)
# #     mask_table_hsv = cv.dilate(mask_table_hsv, kernel, iterations=2)
# #     mask_table_hsv = cv.erode(mask_table_hsv, kernel, iterations=2)
    
# #     nb_components, output, stats, centroids = cv.connectedComponentsWithStats(mask_table_hsv, connectivity=4)    
# #     sizes = stats[:, -1]

# #     max_label = 1
# #     max_size = sizes[1]
# #     for i in range(2, nb_components):
# #         if sizes[i] > max_size:
# #             max_label = i
# #             max_size = sizes[i]

# #     connected_component = np.zeros(output.shape)
# #     connected_component[output == max_label] = 255
# # #     show_image(connected_component)
# #     connected_component = connected_component.astype(np.int8)
# #     table = cv.bitwise_and(frame, frame, mask=connected_component)  
    
# # #     show_image(table)
# #     cv.destroyAllWindows()
# #     return table, connected_component

# def find_table2(frame, show_results=True):
#     low_green = (55, 145, 50)
#     high_green = (75, 255, 255)
#     frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
#     # get the mask for green
#     mask_table_hsv = cv.inRange(frame_hsv, low_green, high_green)

#     kernel = np.ones((15, 15), np.uint8)
#     mask_table_hsv = cv.dilate(mask_table_hsv, kernel, iterations=2)
#     mask_table_hsv = cv.erode(mask_table_hsv, kernel, iterations=2)
       
#     nb_components, output, stats, centroids = cv.connectedComponentsWithStats(mask_table_hsv, connectivity=4)    
#     sizes = stats[:, -1]

#     max_label = 1
#     max_size = sizes[1]
#     for i in range(2, nb_components):
#         if sizes[i] > max_size:
#             max_label = i
#             max_size = sizes[i]

#     connected_component = np.zeros(output.shape)
#     connected_component[output == max_label] = 255
#     #     show_image(connected_component)
#     connected_component = connected_component.astype(np.int8)
#     table = cv.bitwise_and(frame, frame, mask=connected_component)  

#     table = cv.bitwise_and(frame, frame, mask=mask_table_hsv)  
#     contours, hierarchy = cv.findContours(mask_table_hsv, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
#     max_area = 0        
#     img_contour = np.zeros(table.shape, np.uint8)

#     # take the largest in area contour
#     if len(contours) > 0:
#         cnt = contours[0]
#         for i, cont in enumerate(contours):   
#             contour_area = cv.contourArea(cont)            
#             if contour_area > max_area:
#                 max_area = contour_area
#                 cnt = contours[i]     

#         epsilon = 0.01 * cv.arcLength(cnt, True) # contour perimeter
#         approx = cv.approxPolyDP(cnt, epsilon, True) # get the approximation
#         print(cnt)
#         hull = cv.convexHull(cnt)
#         cv.drawContours(table, [approx], 0, (255, 255, 255), -1)
#         table_mask = cv.inRange(table, (255, 255, 255), (255, 255, 255))
#         show_image(table_mask)
        
#         table2 = cv.bitwise_and(frame, frame, mask=table_mask)  
        
#     return table2

# def find_contours(image):
# #     image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
#     table = find_table(image)[0]
#     gray = cv.cvtColor(table, cv.COLOR_BGR2GRAY)
#     ret, thresh = cv.threshold(gray, 50, 255, cv.THRESH_BINARY)
#     contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
    
#     max_area = 0        
#     img_contour = np.zeros(table.shape, np.uint8)

#     # take the largest in area contour
#     if len(contours) > 0:
#         cnt = contours[0]
#         for i, cont in enumerate(contours):   
#             contour_area = cv.contourArea(cont)            
#             if contour_area > max_area:
#                 max_area = contour_area
#                 cnt = contours[i]     

#         epsilon = 0.5 * cv.arcLength(cnt, True) # contour perimeter
#         approx = cv.approxPolyDP(cnt, epsilon, True) # get the approximation
#         hull = cv.convexHull(cnt)
        
#     cv.drawContours(table, [approx], 0, (0, 0, 255), -1)
#     table_gray = cv.cvtColor(table, cv.COLOR_BGR2GRAY)
#     ret2, thresh2 = cv.threshold(table_gray, 50, 255, cv.THRESH_BINARY)
#     table2 = cv.bitwise_and(image, image, mask=thresh2)  
#     show_image(table2)
    

# # def find_contours(frame):
# #     table, mask_table_hsv = find_table(frame)      
# #     contours, _ = cv.findContours(mask_table_hsv, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)   
# #     max_area = 0        
# #     img_contour = np.zeros(table.shape, np.uint8)

# #     # take the largest in area contour
# #     if len(contours) > 0:
# #         cnt = contours[0]
# #         for i, cont in enumerate(contours):   
# #             contour_area = cv.contourArea(cont)            
# #             if contour_area > max_area:
# #                 max_area = contour_area
# #                 cnt = contours[i]     

# #         epsilon = 0.01 * cv.arcLength(cnt, True) # contour perimeter
# #         approx = cv.approxPolyDP(cnt, epsilon, True) # get the approximation
# #         hull = cv.convexHull(cnt)
# #         cv.drawContours(img_contour, [approx], 0, (0, 0, 255), -1)
# #         cv.imshow('img_contour', img_contour) 

# # #     table, balls, mask_balls = detect_balls_using_hsv(table, show_results)    
# # #         cv.imshow('balls', balls)  
# #     cv.destroyAllWindows()

In [157]:
def find_table(frame, show_results=True):
    low_green = (55, 145, 50)
    high_green = (75, 255, 255)
    frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    # get the mask for green
    mask_table_hsv = cv.inRange(frame_hsv, low_green, high_green)

    kernel = np.ones((15, 15), np.uint8)
    mask_table_hsv = cv.dilate(mask_table_hsv, kernel, iterations=2)
    mask_table_hsv = cv.erode(mask_table_hsv, kernel, iterations=2)
       
    nb_components, output, stats, centroids = cv.connectedComponentsWithStats(mask_table_hsv, connectivity=4)    
    sizes = stats[:, -1]

    max_label = 1
    max_size = sizes[1]
    for i in range(2, nb_components):
        if sizes[i] > max_size:
            max_label = i
            max_size = sizes[i]

    connected_component = np.zeros(output.shape)
    connected_component[output == max_label] = 255
    #     show_image(connected_component)
    connected_component = connected_component.astype(np.int8)
    table = cv.bitwise_and(frame, frame, mask=connected_component)  

    table = cv.bitwise_and(frame, frame, mask=mask_table_hsv)  
    contours, hierarchy = cv.findContours(mask_table_hsv, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
    max_area = 0        
    img_contour = np.zeros(table.shape, np.uint8)

    # take the largest in area contour
    if len(contours) > 0:
        cnt = contours[0]
        for i, cont in enumerate(contours):   
            contour_area = cv.contourArea(cont)            
            if contour_area > max_area:
                max_area = contour_area
                cnt = contours[i]     

        epsilon = 0.005 * cv.arcLength(cnt, True) # contour perimeter
        approx = cv.approxPolyDP(cnt, epsilon, True) # get the approximation
#         print(approx)
        hull = cv.convexHull(cnt)
        cv.drawContours(table, [approx], 0, (255, 255, 255), -1)
        table_mask = cv.inRange(table, (255, 255, 255), (255, 255, 255))
#         show_image(table_mask)
        
        table2 = cv.bitwise_and(frame, frame, mask=table_mask)  
        
    return table2

In [158]:
for i in range(0, len(task1)):
    img = read_image(i, task1, 1)
    contour = find_table(img)
#     show_image(contour)

In [159]:
def open_video(path):
    cap = cv.VideoCapture(path)
    
    if cap.isOpened() == False: 
        print("Error opening video stream or file") 
    # Display the video
    current_frame = 0
    max_number_of_frame_to_run = 750

    while cap.isOpened(): 

        ret, frame = cap.read() # Read the frame
        
        if ret is True:
            current_frame = current_frame + 1 
            cv.imshow("Frame", frame)

            if current_frame > max_number_of_frame_to_run:
                break

            if cv.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break

    # after playing the video, release the video capture    
    cap.release()
    # close all the frames
    cv.destroyAllWindows()

def write_video(path):
    # We are going to write the same video at 1 fps, first we need to read the frames.
    frames = read_frames(path)
    # here we have the extensions and the fourcc for each of it
    video_extension_and_fourcc_dict = {'avi': cv.VideoWriter_fourcc('M', 'J', 'P', 'G'),
                                       'mp4': 0x7634706d}   

    # We need to create a VideoWriter object. 
    # First, we should specify the output file name with its format (eg: 1_fps_1.mp4). 
    # we should specify the FourCC code and the number of frames per second (FPS). 
    # Lastly, the frame size should be passed (width, height).

    video_output_name = "Modified/Task2/" + path.split("/")[-1].split('.')[-2] + "_modified.mp4"
    output_video = cv.VideoWriter(video_output_name, video_extension_and_fourcc_dict["mp4"], 30,
                                  (frames[0].shape[1], frames[0].shape[0]))

    num_frames = len(frames)
    # We know that the first video has 30 fps.
    for i in range(0, num_frames):
        output_video.write(frames[i]) # writing the frame

    # don't forget to release the video writer
    output_video.release()

In [9]:
open_video(task2_modified[0])

In [161]:
# print(task2[0].split("/")[-1].split('.')[-2])
write_video(task2[1])

In [3]:
# task 1
# read the templates balls
templates = []
base_folder_matching = 'training_data/template_matching/Task1'
images_names = glob.glob(os.path.join(base_folder_matching, "*.jpg")) 
    
color_dict = {0: "black",
              1: "blue",
              2: "brown",
              3: "green",
              4: "pink",
              5: "red",
              6: "white",
              7: "yellow"}